In [1]:
# Setup plotting
import matplotlib.pyplot as plt

plt.style.use('seaborn-whitegrid')
# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)


import pandas as pd
red_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')

# Create training and validation splits
df_train = red_wine.sample(frac=0.7, random_state=0)
df_valid = red_wine.drop(df_train.index)

# Split features and target
X_train = df_train.drop('quality', axis=1).to_numpy()
X_valid = df_valid.drop('quality', axis=1).to_numpy()
y_train = df_train['quality'].to_numpy()
y_valid = df_valid['quality'].to_numpy()

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow.keras.backend as K

import tensorflow.experimental.numpy as tnp

class BatchNormFCLayer(tf.keras.layers.Layer):  # for the case of fully connected (1D inputs)
    """
    We create class variables such as gamma and beta to be trained. Also creating list of the accumulating
    tensors of moving_mean and moving_variance that are expected to grow up to window size. They are helpful for the
    last part of batchnorm algorithm where another affine transformation is performed using the averages
    """

    def __init__(self, epsilon=0.00000001, window=5):
        """
        :param epsilon:
        :param window:
        """
        super().__init__()

        gamma_init = tf.ones_initializer()
        self.gamma = tf.Variable(
            initial_value=gamma_init(shape=[1], dtype='float32'), trainable=True)
        beta_init = tf.zeros_initializer()
        self.beta = tf.Variable(
            initial_value=beta_init(shape=[1], dtype='float32'), trainable=True)
        self.window = window
        self.epsilon = epsilon
        #self.moving_mean = []
        #self.moving_variance = []
        #self.moving_mean = tf.constant([])
        #self.moving_variance = tf.constant([])
        self.idx = 0
        self.moving_mean = tf.TensorArray(tf.float32, size=0, dynamic_size=True, clear_after_read=False)
        self.moving_variance = tf.TensorArray(tf.float32, size=0, dynamic_size=True, clear_after_read=False)

    def call(self, inputs, training=False):  # Defines the computation from inputs to outputs
        mu = tf.math.reduce_mean(inputs, axis=0)
        variance = K.var(inputs, axis=0)
        
        #outputs = batchnorm_calculations(self, mu, variance, inputs, training)
        
        if training:  # In case of training, perform batch normalization to learn beta and gamma
            x_hat = (inputs - mu) / K.sqrt(variance + self.epsilon)
            outputs = self.gamma * x_hat + self.beta
        else:  # In case of testing - calculation of the inference model
            #self.moving_mean.append(mu)
            #self.moving_variance.append(variance)
            #self.moving_mean = tf.concat([self.moving_mean, mu], 1)
            #self.moving_variance = tf.concat([self.moving_variance, variance], 1)
            self.moving_mean = self.moving_mean.write(self.idx, mu)
            self.moving_variance = self.moving_variance.write(self.idx, variance)
            self.idx = self.idx + 1

            #if len(self.moving_mean) > self.window:  # keep the scope of the window
            if self.idx == self.window:  # keep the scope of the window
                #self.moving_mean = self.moving_mean[1:]
                #self.moving_variance = self.moving_variance[1:]
                self.idx = 0
            #tf.print(self.moving_mean.stack(), self.idx, self.window)

            #if len(self.moving_mean) == 1:  # In case this is the first batch, than the average will be itself
            if self.idx == 1:  # In case this is the first batch, than the average will be itself
                current_mean_means = mu
                current_mean_variances = variance
            else:  # In the regular case we calculate the mean and variance according to the given in the paper
                #tf.print(self.moving_mean.stack())
                #current_mean_means = tf.keras.layers.Average()(self.moving_mean.stack())
                current_mean_means = tf.math.reduce_mean(self.moving_mean.stack(), axis=0)

                # tf.shape()[0], can return a dynamic scalar tensor pointing to inputs' actual batch size. But it return int32
                dim = tf.cast(tf.shape(inputs)[0], dtype=tf.float32) # inputs.shape[0]
                # K.mean() makes 'ListWrapper' object has no attribute 'dtype' error
                # the solution is to use: K.mean(tf.stack(curr_layer.moving_variance), axis=0)
                #testalpha = tf.keras.layers.Average()(self.moving_variance)
                testalpha = tf.math.reduce_mean(self.moving_variance.stack(), axis=0)
                current_mean_variances = dim / (dim - 1) * testalpha

            outputs = self.gamma / K.sqrt(current_mean_variances + self.epsilon) * inputs + \
                      (self.beta - self.gamma * current_mean_means / K.sqrt(current_mean_variances +
                                                                                        self.epsilon))
        
        return outputs




In [3]:
bn = BatchNormFCLayer(window=3)
inp = bn(X_train[:4,:],training=False)
#print(inp)

inp = bn(X_train[4:8,:],training=False)
#print(inp)

inp = bn(X_train[8:12,:],training=False)
#print(inp)

inp = bn(X_train[12:16,:],training=False)
#print(inp)

inp = bn(X_train[16:20,:],training=False)
#print(inp)

inp = bn(X_train[20:24,:],training=False)
#print(inp)

inp = bn(X_train[24:28,:],training=False)
#print(inp)

In [4]:
A = tf.constant([ ])
B = tf.constant([0.1])
out = tf.concat([A, B], 0)
C = tf.constant([0.2])
out = tf.concat([out, C], 0)
print(out) 

tf.Tensor([0.1 0.2], shape=(2,), dtype=float32)


In [5]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=[11]),
    #layers.Dropout(0.3),
    BatchNormFCLayer(),
    layers.Dense(1024, activation='relu'),
    #layers.Dropout(0.3),
    #BatchNormFCLayer(),
    #layers.Dense(1024, activation='relu'),
    #layers.Dropout(0.3),
    #BatchNormFCLayer(),
    layers.Dense(1),
])

In [6]:
model.compile(
    optimizer='adam',
    loss='mae',
)

history = model.fit(
    X_train, y_train,
    batch_size=256,
    epochs=100,
    validation_data=(X_valid, y_valid),
    verbose=0
)


# Show the learning curves
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot();

TypeError: ignored

In [ ]:
import numpy as np

inp = np.array([[[1., 1., 1., 1.],
                [1., 1., 1., 1.],
                [1., 1., 1., 1.]],
               [[1., 2., 3., 4.],
                [1., 2., 3., 4.],
                [1., 2., 3., 4.]]])